In [51]:
import time
import sys
import numpy

import dataset_walker
import ontology_reader
from baseline import BaselineMethod1, BaselineMethod2

In [52]:
DATASET = 'dstc5_dev'
DATAROOT = '../data'
TRACKFILE = 'baseline_dev.json'
ONTOLOGY = 'config/ontology_dstc5.json'
METHOD = '1'

dataset = dataset_walker.dataset_walker(DATASET, dataroot=DATAROOT, labels=False, translations=True)

track = {"sessions": []}
track["dataset"] = DATASET
start_time = time.time()

ontology = ontology_reader.OntologyReader(ONTOLOGY)

for call in dataset:
    this_session = {"session_id":call.log["session_id"], "utterances":[]}
    tracker.reset()
    for (utter, translations, _) in call:
        print translations
        break
        tracker_result = tracker.addUtter(utter, translations)
        if tracker_result is not None:
            this_session["utterances"].append(tracker_result)
    track["sessions"].append(this_session)

{u'translated': [{u'hyp': u'Hello .', u'align': [[u'\u4f60\u597d', [0]], [u'.', [1]]]}, {u'hyp': u'how do you do .', u'align': [[u'\u4f60', [1, 2, 3]], [u'\u597d', [0]], [u'.', [4]]]}, {u'hyp': u'how do you do ?', u'align': [[u'\u4f60', [1, 2, 3]], [u'\u597d', [0]], [u'.', [4]]]}, {u'hyp': u'you are good .', u'align': [[u'\u4f60', [0, 1]], [u'\u597d', [2]], [u'.', [3]]]}, {u'hyp': u'do you good .', u'align': [[u'\u4f60', [0, 1]], [u'\u597d', [2]], [u'.', [3]]]}], u'utter_index': 0}
{u'translated': [{u'hyp': u'Hello ?', u'align': [[u'\u5582', [0]], [u'?', [1]]]}, {u'hyp': u'hey ?', u'align': [[u'\u5582', [0]], [u'?', [1]]]}, {u'hyp': u'hello ? "', u'align': [[u'\u5582', [0]], [u'?', [1]]]}, {u'hyp': u'hello .', u'align': [[u'\u5582', [0]], [u'?', [1]]]}, {u'hyp': u'hi there ?', u'align': [[u'\u5582', [0, 1]], [u'?', [2]]]}], u'utter_index': 0}


In [53]:
def print_dataset_info(in_dataset):
    print 'Number of dialogs:\t{}'.format(len(dataset))
    print 'Mean dialog length (turns):\t{}'.format(numpy.mean([len(dialog) for dialog in dataset]))

In [63]:
def print_ontology_info(in_ontology):
    print 'Number of topics: {}'.format(len(in_ontology.get_topics()))
    print ''

DSTC 5 Training Set Info
==

In [60]:
print_dataset_info(dataset)

Number of dialogs:	2
Mean dialog length (turns):	1565.0


In [65]:
print dir(ontology)
print_ontology_info(ontology)

['__doc__', '__init__', '__module__', 'get_pilot_tagsets', 'get_slots', 'get_tagsets', 'get_topics', 'get_translated_tagsets', 'get_translations', 'ontology', 'pilot_tagsets', 'tagsets', 'translations']
Number of topics: 5
